In [1]:
import os
import sys

def is_colab():
    return 'google.colab' in sys.modules

if is_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    PROJECT_PATH = '/content/drive/MyDrive/NBS_Project'
else:
    # 自动根据当前脚本定位项目根路径
    PROJECT_PATH = os.path.abspath(os.path.join(os.path.dirname(__file__), '..')) \
        if '__file__' in globals() else os.getcwd()

if PROJECT_PATH not in sys.path:
    sys.path.append(PROJECT_PATH)

print(f"项目路径设置为: {PROJECT_PATH}")

项目路径设置为: c:\Users\a1831\Desktop\NBS_Project


In [2]:
from src.config import Config
from src.main import create_MDs,create_ESP
import src.baseline as baseline
import src.NBS as NBS
import src.model as model
import numpy as np

CONFIG_PATH = os.path.join(PROJECT_PATH, "configs", "test.json")
config = Config(CONFIG_PATH)

# Create MDs with a specific seed for reproducibility
seed = 41
MDs = create_MDs(config, seed=seed)
# np.random.shuffle(MDs)  # Shuffle the MDs

for md in MDs:
    print(md.param)

esp = create_ESP(config, seed=seed)
print(esp.param)

{'s': 0.1, 'l': 0.8, 'cn': 0.32332202513299674, 'Fn': 1.184383282690447, 'kn': 0.6732799162883472, 'omega_n': 1, 'Rn': 4.13472934445215}
{'s': 0.1, 'l': 0.8, 'cn': 0.6374404631644437, 'Fn': 1.7637226511189565, 'kn': 0.6651454029151388, 'omega_n': 1, 'Rn': 10.248845012378245}
{'s': 0.1, 'l': 0.8, 'cn': 0.395711266542491, 'Fn': 2.1321345406869066, 'kn': 0.19255662205025992, 'omega_n': 1, 'Rn': 6.716553285779903}
{'s': 0.1, 'l': 0.8, 'cn': 0.16187321635232887, 'Fn': 3.8199302863134226, 'kn': 0.3183833952999494, 'omega_n': 1, 'Rn': 9.231363952354403}
{'s': 0.1, 'l': 0.8, 'cn': 0.6413215489755134, 'Fn': 3.913825949541608, 'kn': 0.4233228768151258, 'omega_n': 1, 'Rn': 7.948101841125762}
{'s': 0.1, 'l': 0.8, 'cn': 0.49308149762923636, 'Fn': 2.4920855793255217, 'kn': 0.5819339189335254, 'omega_n': 1, 'Rn': 3.859406010253179}
{'s': 0.1, 'l': 0.8, 'cn': 0.17404615893711323, 'Fn': 1.5048957788256199, 'kn': 0.3264336993988291, 'omega_n': 1, 'Rn': 7.130152926250668}
{'s': 0.1, 'l': 0.8, 'cn': 0.350

In [3]:
T = 100
N = len(MDs)
lam_uni, p_uni, r_uni, Dmax_uni = np.zeros(N),np.zeros(N),np.zeros(N),0
for _ in range(T):
    l,p,r,d = baseline.uniform_baseline(esp, MDs,43)
    lam_uni += l
    p_uni += p 
    r_uni += r
    Dmax_uni += d
lam_uni, p_uni, r_uni, Dmax_uni = lam_uni/T, p_uni/T, r_uni/T, Dmax_uni/T

print(f"Uniform λ: {lam_uni}")
print(f"Uniform p: {p_uni}")  
print(f"Uniform Dmax: {Dmax_uni}")
print(f"Uniform r: {r_uni}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("sum lambda:", np.sum(lam_uni), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_uni[i],lam_uni[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_uni, " D0:", esp.D0)
max_Dn = max(md.delay(p_uni[i], lam_uni[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_uni == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_uni[i], lam_uni[i]) <= Dmax_uni for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_uni <= esp.D0)
print("pn<=Fn:", all(p_uni[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_uni[i] <= p_uni[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_uni), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_uni[i] >= md.Ln(p_uni[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_uni[i]) for i,md in enumerate(MDs)]))

Uniform λ: [5. 5. 5. 5. 5. 5. 5. 5. 5. 5.]
Uniform p: [0.92100001 0.48527993 0.46095682 3.27435173 2.47417436 0.89095722
 1.23730207 1.19202808 1.87957382 2.17605924]
Uniform Dmax: 1.3272929506848081
Uniform r: [23.04668976 23.04668976 23.04668976 23.04668976 23.04668976 23.04668976
 23.04668976 23.04668976 23.04668976 23.04668976]
social welfare: 230.4668976068324
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.17773623972334668', 'MD 2 Dn: 0.9478442742383153', 'MD 3 Dn: 1.3272929506848183', 'MD 4 Dn: 0.03866499814551409', 'MD 5 Dn: 0.05115118397187841', 'MD 6 Dn: 0.18885771097720916', 'MD 7 Dn: 0.10956991337392175', 'MD 8 Dn: 0.11098137321648126', 'MD 9 Dn: 0.06614255302124675', 'MD 10 Dn: 0.06251306644019254']
Dmax: 1.3272929506848081  D0: 3
max Dn: 1.3272929506848183
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 230.4668976068324 Q_star: 247.01083342594447
rn>=Ln: True
sum Ln: 16.543935819112093


In [4]:
lam_pro, p_pro, r_pro, Dmax_pro = baseline.proportional_baseline(esp,MDs,43)
print(f"Uniform λ: {lam_pro}")
print(f"Uniform p: {p_pro}")  
print(f"Uniform Dmax: {Dmax_pro}")
print(f"Uniform r: {r_pro}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("sum lambda:", np.sum(lam_pro), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_pro[i],lam_pro[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_pro, " D0:", esp.D0)
max_Dn = max(md.delay(p_pro[i], lam_pro[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_pro == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_pro[i], lam_pro[i]) <= Dmax_pro for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_pro <= esp.D0)
print("pn<=Fn:", all(p_pro[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_pro[i] <= p_pro[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_pro), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_pro[i] >= md.Ln(p_pro[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_pro[i]) for i,md in enumerate(MDs)]))

Uniform λ: [2.09025991 3.11270752 3.76289958 6.74160743 6.90731927 4.39815949
 2.65591668 5.8750097  6.85821515 7.59790528]
Uniform p: [0.8400625112602841, 0.34090587831472074, 0.36397107757518515, 3.2967540152184913, 2.5371703641888748, 0.8536289147195153, 1.1907565012107326, 1.2435702010724685, 1.965798290302121, 2.290152720278999]
Uniform Dmax: 1.2859583266526118
Uniform r: [ 9.62393296 14.33146583 17.32506713 31.03957436 31.80254154 20.24991816
 12.22831857 27.0496024  31.57645732 34.98212386]
social welfare: 230.20900211930916
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 0.14308407152681338', 'MD 2 Dn: 0.8803702081765077', 'MD 3 Dn: 1.2859583266526118', 'MD 4 Dn: 0.03984520611344912', 'MD 5 Dn: 0.05289231865318548', 'MD 6 Dn: 0.18534436514671376', 'MD 7 Dn: 0.0958008585011212', 'MD 8 Dn: 0.11339155745548526', 'MD 9 Dn: 0.06852461299682983', 'MD 10 Dn: 0.06502289420130582']
Dmax: 1.2859583266526118  D0: 3
max Dn: 1.2859583266526118
Dmax == max Dn: True
Dn<=Dmax: True
Dmax<=D0: True
pn<

In [5]:
lam_non, p_non, r_non, Dmax_non = baseline.stackelberg_br(esp,MDs)
print(f"Non-cooperative λ: {lam_non}")
print(f"Non-cooperative p: {p_non}")  
print(f"Non-cooperative Dmax: {Dmax_non}")
print(f"Non-cooperative r: {r_non}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_non, p_non))
print("sum lambda:", np.sum(lam_non), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_non[i],lam_non[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_non, " D0:", esp.D0)
max_Dn = max(md.delay(p_non[i], lam_non[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_non == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_non[i], lam_non[i]) <= Dmax_non for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_non <= esp.D0)
print("pn<=Fn:", all(p_non[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_non[i] <= p_non[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_non), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_non[i] >= md.Ln(p_non[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_non[i]) for i,md in enumerate(MDs)]))

iteration 0
✅ 这个点严格满足所有约束（在容差范围内）。


c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


iteration 1
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 2
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 3
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 4
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 5
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 6
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 7
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 8
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 9
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 10
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 11
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 12
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 13
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 14
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 15
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 16
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 17
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 18
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 19
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 20
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 21
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 22
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 23
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 24
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 25
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 26
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 27
✅ 这个点严格满足所有约束（在容差范围内）。
iteration 28
✅ 这个点严格满足所有约束（在容差范围内）。
i

In [6]:
lam_swm, p_swm, Dmax_swm = baseline.social_welfare_maximization(esp, MDs)
print(f"SWM λ: {lam_swm}")
print(f"SWM p: {p_swm}")  
print(f"SWM Dmax: {Dmax_swm}")
r_swm = baseline.solve_r_NBP(esp, MDs, Dmax_swm, lam_swm, p_swm)
print(f"SWM r: {r_swm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("sum lambda:", np.sum(lam_swm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_swm[i],lam_swm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_swm, " D0:", esp.D0)
max_Dn = max(md.delay(p_swm[i], lam_swm[i]) for i, md in enumerate(MDs))
print("max Dn:", max_Dn)
print("Dmax == max Dn:", Dmax_swm == max_Dn)
print("Dn<=Dmax:", all(md.delay(p_swm[i], lam_swm[i]) <= Dmax_swm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_swm <= esp.D0)
print("pn<=Fn:", all(p_swm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_swm[i] <= p_swm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_swm), "Q_star:", esp.Q(max_Dn))
print("rn>=Ln:", all(r_swm[i] >= md.Ln(p_swm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_swm[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。


c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


Optimization terminated successfully    (Exit mode 0)
            Current function value: -243.3480999047845
            Iterations: 316
            Function evaluations: 7249
            Gradient evaluations: 316
SWM λ: [3.07863735 2.9162451  6.54226158 8.40311345 5.48324198 3.02016642
 9.41095494 1.75177419 7.60059485 1.79301014]
SWM p: [0.34633578 0.33157126 0.62227595 0.77065072 0.53727315 0.34187444
 0.85166595 0.23843987 0.70659958 0.24265237]
SWM Dmax: 0.8238281718887738
Optimization terminated successfully    (Exit mode 0)
            Current function value: -37.589656695490305
            Iterations: 12
            Function evaluations: 133
            Gradient evaluations: 12
SWM r: [20.70045783 20.68758208 20.65645702 20.63105094 20.72193568 20.62645879
 20.82758921 20.62501875 20.74497368 20.62340831]
social welfare: 245.14588308480256
sum lambda: 49.99999999999999  lambda0: 50
['MD 1 Dn: 0.8238271719158742', 'MD 2 Dn: 0.8238271719106848', 'MD 3 Dn: 0.8238271719160407', 'MD

In [7]:
lam_opt, p_opt, r_opt, Dmax_opt = baseline.optimal_NBP(esp, MDs)
print(f"Optimal λ: {lam_opt}")
print(f"Optimal p: {p_opt}")  
print(f"Optimal r: {r_opt}")
print(f"Optimal Dmax: {Dmax_opt}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("sum lambda:", np.sum(lam_opt), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_opt[i],lam_opt[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_opt, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_opt == max(md.delay(p_opt[i], lam_opt[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_opt[i], lam_opt[i]) <= Dmax_opt for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_opt <= esp.D0)
print("pn<=Fn:", all(p_opt[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lam_opt[i] <= p_opt[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_opt), "Q_star:", esp.Q(Dmax_opt))
print("rn>=Ln:", all(r_opt[i] >= md.Ln(p_opt[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_opt[i]) for i,md in enumerate(MDs)]))

✅ 这个点严格满足所有约束（在容差范围内）。
Optimization terminated successfully    (Exit mode 0)
            Current function value: -36.08592227690471
            Iterations: 62
            Function evaluations: 1988
            Gradient evaluations: 62
Optimal λ: [4.84457357 4.76190954 5.17122479 5.2986799  4.99066676 4.93821108
 5.18234376 4.82700188 5.1440049  4.84138383]
Optimal p: [0.44866472 0.44138567 0.47436606 0.48437644 0.45981547 0.45623634
 0.47521587 0.446603   0.4720592  0.44809776]
Optimal r: [20.93521751 20.94120874 20.70640869 20.66545284 20.7900626  20.85433209
 20.73518339 20.96407123 20.7269886  20.95544657]
Optimal Dmax: 1.3335392265482438
social welfare: 246.75258752284773
sum lambda: 50.0  lambda0: 50
['MD 1 Dn: 1.3335392265565387', 'MD 2 Dn: 1.3335392265443413', 'MD 3 Dn: 1.3335392265523667', 'MD 4 Dn: 1.333539226548163', 'MD 5 Dn: 1.3335392265498447', 'MD 6 Dn: 1.3335392265510366', 'MD 7 Dn: 1.333539226551363', 'MD 8 Dn: 1.3335392265515211', 'MD 9 Dn: 1.3335392265516688', 'MD 10 

In [8]:
lamb_admm, p_admm, Dmax_admm = NBS.ADMM(esp, MDs)
print(f"ADMM λ: {lamb_admm}")
print(f"ADMM p: {p_admm}")  
print(f"ADMM Dmax: {Dmax_admm}")
r_admm = NBS.negotiation(esp, MDs,lamb_admm, p_admm, Dmax_admm)
print(f"ADMM r: {r_admm}")
print("==============================")
print("social welfare:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("sum lambda:", np.sum(lamb_admm), " lambda0:", esp.lambda0)
print([f"MD {i+1} Dn: {md.delay(p_admm[i],lamb_admm[i])}" for i, md in enumerate(MDs)])
print("Dmax:", Dmax_admm, " D0:", esp.D0)
print("max Dn:", max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dmax == max Dn:", Dmax_admm == max(md.delay(p_admm[i], lamb_admm[i]) for i, md in enumerate(MDs)))
print("Dn<=Dmax:", all(md.delay(p_admm[i], lamb_admm[i]) <= Dmax_admm for i, md in enumerate(MDs)))
print("Dmax<=D0:", Dmax_admm <= esp.D0)
print("pn<=Fn:", all(p_admm[i] <= md.Fn for i, md in enumerate(MDs)))
print("lambda_n<=pn/sl:", all(lamb_admm[i] <= p_admm[i] / (md.s*md.l) for i, md in enumerate(MDs)))
print("sum r:", np.sum(r_admm), "Q_star:", esp.Q(Dmax_admm))
print("rn>=Ln:", all(r_admm[i] >= md.Ln(p_admm[i]) for i, md in enumerate(MDs)))
print("sum Ln:", np.sum([md.Ln(p_admm[i]) for i,md in enumerate(MDs)]))

c:\Users\a1831\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


ADMM λ: [3.76289398 3.37432012 6.23207938 7.63261639 4.8664029  4.47928344
 6.4456136  3.5344267  6.02616197 3.64620152]
ADMM p: [0.36252946 0.33076909 0.55962754 0.67148169 0.45026631 0.41992222
 0.57667002 0.34358786 0.54302338 0.35287833]
ADMM Dmax: 1.3250467238690902
ADMM r: [20.8703184  20.84133503 20.77321249 20.7478578  20.80327111 20.84357382
 20.80803637 20.87932046 20.78874217 20.8804834 ]
social welfare: 247.00088075105776
sum lambda: 50.000000000000014  lambda0: 50
['MD 1 Dn: 1.325042030137971', 'MD 2 Dn: 1.3250386908839524', 'MD 3 Dn: 1.3250497712606535', 'MD 4 Dn: 1.3250576901396205', 'MD 5 Dn: 1.3250450615189089', 'MD 6 Dn: 1.3250434847150911', 'MD 7 Dn: 1.325050487738365', 'MD 8 Dn: 1.3250348242078096', 'MD 9 Dn: 1.3250460364213215', 'MD 10 Dn: 1.325039205712268']
Dmax: 1.3250467238690902  D0: 3
max Dn: 1.3250576901396205
Dmax == max Dn: False
Dn<=Dmax: False
Dmax<=D0: True
pn<=Fn: True
lambda_n<=pn/sl: True
sum r: 208.23615105516 Q_star: 249.40296842052217
rn>=Ln: True

In [10]:
print("Nash Product Opt:",model.nash_product_log(esp, MDs, lam_opt, p_opt, r_opt))
print("Nash Product SWM:",model.nash_product_log(esp, MDs, lam_swm, p_swm, r_swm))
print("Nash Product ADMM:",model.nash_product_log(esp, MDs, lamb_admm, p_admm, r_admm))
print("Nash Product Proportional:",model.nash_product_log(esp, MDs, lam_pro, p_pro, r_pro))
print("Nash Product Uniform:",model.nash_product_log(esp, MDs, lam_uni, p_uni, r_uni))
print("Nash Product Non-cooperative:",model.nash_product_log(esp, MDs, lam_non, p_non, r_non))
print("=============================")
print("SW Opt:",model.social_welfare(esp, MDs, lam_opt, p_opt))
print("SW SWM:",model.social_welfare(esp, MDs, lam_swm, p_swm))
print("SW ADMM:",model.social_welfare(esp, MDs, lamb_admm, p_admm))
print("SW Proportional:",model.social_welfare(esp, MDs, lam_pro, p_pro))
print("SW Uniform:",model.social_welfare(esp, MDs, lam_uni, p_uni))
print("SW Non-cooperative:",model.social_welfare(esp, MDs, lam_non, p_non))
print("=============================")
print("Dmax Opt:", Dmax_opt)
print("Dmax SWM:", Dmax_swm)
print("Dmax ADMM:", Dmax_admm)
print("Dmax Proportional:", Dmax_pro)
print("Dmax Uniform:", Dmax_uni)
print("Dmax Non-cooperative:", Dmax_non)

Nash Product Opt: 37.66804853967297
Nash Product SWM: 37.686936700989506
Nash Product ADMM: 37.6801173926484
Nash Product Proportional: 35.56375933362734
Nash Product Uniform: 36.333978960125755
Nash Product Non-cooperative: nan
SW Opt: 246.75258752284773
SW SWM: 247.14128523903418
SW ADMM: 247.00088075105776
SW Proportional: 232.5426676667683
SW Uniform: 232.85823086607678
SW Non-cooperative: -454776.198521893
Dmax Opt: 1.3335392265482438
Dmax SWM: 1.319017276373108
Dmax ADMM: 1.3250467238690902
Dmax Proportional: 1.2859583266526118
Dmax Uniform: 1.3272929506848081
Dmax Non-cooperative: 2.999999
